<a href="https://colab.research.google.com/github/AnimeshPandey123/AI-assesment/blob/main/Week4_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mouryarahul/7CS107_PracticalWorks_Assignment/blob/master/Week4_Assignment.ipynb)

# 7CS107 – Week 4 Programming Assignment (100 marks)

**Topic:** ML Fundamentals – Linear Models, ERM, MLE/MAP, Regularization, Polynomial Features, Weighted Least Squares.

Allowed: numpy, scipy, sklearn. Edit only `# TODO` blocks. Run tests after each question.

In [1]:
import numpy as np
from numpy.linalg import inv, lstsq
from scipy import stats
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

np.random.seed(7)
print('Imports OK. NumPy:', np.__version__)


Imports OK. NumPy: 2.0.2


## Synthetic dataset
One synthetic regression dataset with an intercept included.

In [2]:
n, d = 300, 5
X_raw = np.random.randn(n, d)
true_w = np.array([2.0, -1.5, 0.0, 3.0, 0.5, -2.0])
X = np.c_[np.ones((n,1)), X_raw]
sigma = 0.8
noise = np.random.randn(n) * sigma
y = X @ true_w + noise
print('Shapes:', X.shape, y.shape)


Shapes: (300, 6) (300,)


---
### Q1 (10 marks): Empirical Risk via Train/Test Split
Implement `erm_split_mse(X, y, test_size, random_state)` using `train_test_split`, `LinearRegression`, and `mean_squared_error` functions available in sk-learn package. Return `(mse_train, mse_test)`.

In [3]:
def erm_split_mse(X, y, test_size=0.25, random_state=42):
    """Return (mse_train, mse_test) using LinearRegression on a train/test split.
    """
    # TODO: split, fit, predict, compute MSEs
    # raise NotImplementedError()
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )

    # Train model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Predict
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    # Compute MSE
    mse_train = mean_squared_error(y_train, y_pred_train)
    mse_test = mean_squared_error(y_test, y_pred_test)
    return mse_train, mse_test



In [4]:
# Tests for Q1
try:
    mse_tr, mse_te = erm_split_mse(X, y, test_size=0.2, random_state=0)
except NotImplementedError:
    print('Q1 not implemented yet.')
else:
    assert isinstance(mse_tr, float) and isinstance(mse_te, float)
    assert 0 <= mse_tr < 10 and 0 <= mse_te < 15
    print('Q1 passed.')


Q1 passed.


---
### Q2 (15 marks): Model Capacity with Polynomial Features
Implement `poly_model_selection(X_raw, y, degrees)` with `PolynomialFeatures(include_bias=True)`, 80/20 split (`random_state=0`), and `LinearRegression`. Return `(train_mse_list, test_mse_list, best_degree)` minimizing test MSE.

In [5]:
def poly_model_selection(X_raw, y, degrees=(1,2,3,4,5)):
    """Return (train_mse_list, test_mse_list, best_degree).
    """
    # TODO
    X_train, X_test, y_train, y_test = train_test_split(X_raw, y, test_size=0.2, random_state=0)

    train_mse_list = []
    test_mse_list = []

    for d in degrees:
        # Create polynomial features
        poly = PolynomialFeatures(degree=d, include_bias=True)
        X_train_poly = poly.fit_transform(X_train)
        X_test_poly = poly.transform(X_test)

        # Fit linear regression
        model = LinearRegression()
        model.fit(X_train_poly, y_train)

        # Compute MSE
        y_train_pred = model.predict(X_train_poly)
        y_test_pred = model.predict(X_test_poly)
        train_mse = mean_squared_error(y_train, y_train_pred)
        test_mse = mean_squared_error(y_test, y_test_pred)

        train_mse_list.append(train_mse)
        test_mse_list.append(test_mse)

    # Select degree with lowest test MSE
    best_degree = degrees[np.argmin(test_mse_list)]

    return train_mse_list, test_mse_list, best_degree


In [6]:
# Tests for Q2
try:
    tr_mse, te_mse, best_deg = poly_model_selection(X_raw, y, degrees=(1,2,3,4,5))
except NotImplementedError:
    print('Q2 not implemented yet.')
else:
    tr_mse = np.asarray(tr_mse); te_mse = np.asarray(te_mse)
    assert tr_mse.shape == (5,) and te_mse.shape == (5,)
    assert all(m > 0 for m in tr_mse) and all(m > 0 for m in te_mse)
    assert best_deg in (1,2,3,4,5)
    print('Q2 passed. Best degree:', best_deg)


Q2 passed. Best degree: 2


---
### Q3 (15 marks): Closed-form Ordinary Least-Square (Normal Equations)
Implement `ols_fit(X, y)` using `np.linalg.lstsq` (or equivalent). Return `w` of shape `(n_features,)`.

In [7]:
def ols_fit(X, y):
    """Return OLS solution w (shape: (n_features,)).
    """
    # TODO
    w, residuals, rank, s = np.linalg.lstsq(X, y, rcond=None)

    return w


In [8]:
# Tests for Q3
try:
    w_hat = ols_fit(X, y)
except NotImplementedError:
    print('Q3 not implemented yet.')
else:
    assert w_hat.shape == (X.shape[1],)
    lr = LinearRegression(fit_intercept=False).fit(X, y)
    w_sk = lr.coef_
    assert np.allclose(w_hat, w_sk, atol=1e-6)
    print('Q3 passed.')


Q3 passed.


---
### Q4 (15 marks): Closed-form Ridge Regression
Implement `ridge_fit(X, y, lam)` returning $(X^T X + \lambda I)^{-1} X^T y$ without penalizing the intercept (first coefficient).

In [9]:
def ridge_fit(X, y, lam=1.0):
    """Return ridge solution with no penalty on intercept (index 0).
    """
    # TODO
    n_features = X.shape[1]

    # Identity matrix for regularization
    I = np.eye(n_features)

    # Do not penalize intercept (first coefficient)
    I[0, 0] = 0

    # Closed-form solution
    w = np.linalg.inv(X.T @ X + lam * I) @ X.T @ y

    return w


In [10]:
# Tests for Q4
try:
    w_ridge = ridge_fit(X, y, lam=1.25)
except NotImplementedError:
    print('Q4 not implemented yet.')
else:
    d = X.shape[1]
    I = np.eye(d); I[0,0] = 0.0
    w_ref = np.linalg.solve(X.T @ X + 1.25 * I, X.T @ y)
    assert np.allclose(w_ridge, w_ref, atol=1e-6)
    print('Q4 passed.')


Q4 passed.


---
### Q5 (15 marks): MLE for Noise Variance and Negative Log-Likelihood
Implement `mle_sigma2_and_nll(X, y, w)` returning `(sigma2_mle, nll)` under Gaussian likelihood.

In [11]:
def mle_sigma2_and_nll(X, y, w):
    """Return (sigma2_mle, nll) under Gaussian likelihood.
    """
    # TODO
    n = X.shape[0]

    # Residuals
    residuals = y - X @ w

    # MLE estimate of variance
    sigma2_mle = np.sum(residuals**2) / n

    # Negative log-likelihood
    nll = (n / 2) * np.log(2 * np.pi * sigma2_mle) + (1 / (2 * sigma2_mle)) * np.sum(residuals**2)

    return sigma2_mle, nll


In [12]:
# Tests for Q5
try:
    w_hat = ols_fit(X, y)
    s2, nll = mle_sigma2_and_nll(X, y, w_hat)
except NotImplementedError:
    print('Q5 not implemented yet.')
else:
    assert s2 > 0 and np.isfinite(s2)
    resid = y - X @ w_hat
    s2_ref = np.mean(resid**2)
    assert np.allclose(s2, s2_ref, rtol=1e-6)
    assert np.isfinite(nll)
    print('Q5 passed.')


Q5 passed.


---
### Q6 (15 marks): MAP with Gaussian Prior (Equivalence to Ridge)
Implement `map_estimate(X, y, sigma2, tau2)` with no penalty on the intercept; check equivalence to ridge with `lambda = sigma2/tau2`.

In [13]:
def map_estimate(X, y, sigma2, tau2):
    """Return MAP estimate under Gaussian prior with variance tau2 and noise sigma2.
    """
    # TODO
    n_features = X.shape[1]

    # Equivalent ridge lambda
    lam = sigma2 / tau2

    # Identity matrix for regularization
    I = np.eye(n_features)

    # Do not penalize intercept
    I[0, 0] = 0

    # MAP closed-form solution
    w_map = np.linalg.inv(X.T @ X + lam * I) @ X.T @ y

    return w_map


In [14]:
# Tests for Q6
try:
    lam = 1.25
    sigma2 = 0.5
    tau2 = sigma2 / lam
    w_map = map_estimate(X, y, sigma2=sigma2, tau2=tau2)
except NotImplementedError:
    print('Q6 not implemented yet.')
else:
    d = X.shape[1]
    I = np.eye(d); I[0,0] = 0.0
    w_ref = np.linalg.solve(X.T @ X + lam * I, X.T @ y)
    assert np.allclose(w_map, w_ref, atol=1e-6)
    print('Q6 passed.')


Q6 passed.


---
### Q7 (15 marks): Weighted Least Squares (Heteroscedastic Noise)
Implement `wls_fit(X, y, weights)` solving $(X^T W X)^{-1} X^T W y$ with positive weights. Verify with `LinearRegression(..., sample_weight=weights)`.

In [15]:
def wls_fit(X, y, weights):
    """Return Weighted Least Squares solution using (X^T W X)^{-1} X^T W y.
    """
    # TODO
    # raise NotImplementedError()
    # Ensure weights are positive
    assert np.all(weights > 0), "All weights must be positive."

    # Construct diagonal weight matrix
    W = np.diag(weights)

    # Closed-form WLS solution
    w = np.linalg.inv(X.T @ W @ X) @ X.T @ W @ y

    return w



In [16]:
# Tests for Q7
try:
    rng = np.random.default_rng(123)
    n2, d2 = 200, 5
    Xh_raw = rng.normal(size=(n2, d2))
    Xh = np.c_[np.ones((n2,1)), Xh_raw]
    sigma_i = np.linspace(0.3, 1.5, n2)
    eps = rng.normal(scale=sigma_i)
    yh = Xh @ true_w + eps
    weights = 1.0/(sigma_i**2)
    w_wls = wls_fit(Xh, yh, weights)
except NotImplementedError:
    print('Q7 not implemented yet.')
else:
    lr = LinearRegression(fit_intercept=False)
    lr.fit(Xh, yh, sample_weight=weights)
    w_sk = lr.coef_
    assert np.allclose(w_wls, w_sk, atol=1e-6)
    print('Q7 passed.')


Q7 passed.


---
## Auto-scoring (local)
Run to compute a provisional score out of 100.

In [18]:
score = 0

# Q1
try:
    mse_tr, mse_te = erm_split_mse(X, y, test_size=0.2, random_state=0)
    assert isinstance(mse_tr, float) and isinstance(mse_te, float)
    assert 0 <= mse_tr < 10 and 0 <= mse_te < 15
    score += 10
except Exception as e:
    print('Q1 failed:', e)

# Q2
try:
    tr_mse, te_mse, best_deg = poly_model_selection(X_raw, y, degrees=(1,2,3,4,5))
    tr_mse = np.asarray(tr_mse); te_mse = np.asarray(te_mse)
    assert tr_mse.shape == (5,) and te_mse.shape == (5,)
    assert all(m > 0 for m in tr_mse) and all(m > 0 for m in te_mse)
    assert best_deg in (1,2,3,4,5)
    score += 15
except Exception as e:
    print('Q2 failed:', e)

# Q3
try:
    w_hat = ols_fit(X, y)
    lr = LinearRegression(fit_intercept=False).fit(X, y)
    w_sk = lr.coef_
    assert np.allclose(w_hat, w_sk, atol=1e-6)
    score += 15
except Exception as e:
    print('Q3 failed:', e)

# Q4
try:
    lam = 1.25
    I = np.eye(X.shape[1]); I[0,0] = 0.0
    w_ref = np.linalg.solve(X.T @ X + lam * I, X.T @ y)
    w_ridge = ridge_fit(X, y, lam=lam)
    assert np.allclose(w_ridge, w_ref, atol=1e-6)
    score += 15
except Exception as e:
    print('Q4 failed:', e)

# Q5
try:
    w_hat = ols_fit(X, y)
    s2, nll = mle_sigma2_and_nll(X, y, w_hat)
    resid = y - X @ w_hat
    s2_ref = np.mean(resid**2)
    assert np.allclose(s2, s2_ref, rtol=1e-6)
    assert np.isfinite(nll)
    score += 15
except Exception as e:
    print('Q5 failed:', e)

# Q6
try:
    lam = 1.25
    sigma2 = 0.5
    tau2 = sigma2 / lam
    w_map = map_estimate(X, y, sigma2=sigma2, tau2=tau2)
    I = np.eye(X.shape[1]); I[0,0] = 0.0
    w_ref = np.linalg.solve(X.T @ X + lam * I, X.T @ y)
    assert np.allclose(w_map, w_ref, atol=1e-6)
    score += 15
except Exception as e:
    print('Q6 failed:', e)

# Q7
try:
    rng = np.random.default_rng(123)
    n2, d2 = 200, X.shape[1]-1
    Xh_raw = rng.normal(size=(n2, d2))
    Xh = np.c_[np.ones((n2,1)), Xh_raw]
    sigma_i = np.linspace(0.3, 1.5, n2)
    eps = rng.normal(scale=sigma_i)
    yh = Xh @ true_w + eps
    weights = 1.0/(sigma_i**2)
    w_wls = wls_fit(Xh, yh, weights)
    lr = LinearRegression(fit_intercept=False)
    lr.fit(Xh, yh, sample_weight=weights)
    w_sk = lr.coef_
    assert np.allclose(w_wls, w_sk, atol=1e-6)
    score += 15
except Exception as e:
    print('Q7 failed:', e)

print('Provisional score:', score, '/ 100')


Provisional score: 100 / 100
